In [6]:
import cv2
import numpy as np
import skimage.io as io
from skimage.io import imread
from skimage.io import imread_collection
from skimage.io import imshow_collection
from skimage.util import img_as_ubyte
from skimage import measure
from skimage import feature
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from stl import mesh


#__USER'S INITIALIZATIONS______________________________________________#
default_name_input = "20--100.tif"
default_name_output = "trial.stl"
#__NON-USER'S INITIALIZATIONS__________________________________________#
name_of_input = default_name_input
#name_of_output = default_name_output
images = img_as_ubyte(io.imread(name_of_input))
print('The shape of images is ', images.shape)
print('The type of image is ', images.dtype)

The shape of images is  (81, 1977, 1849)
The type of image is  uint8


In [7]:
#__FUNCTION(S)_________________________________________________________#

def fill_holes (to_fill_img, size_ellipse) :
    
    # Dilation of the image
    kern = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size_ellipse,size_ellipse))
    #kern = np.ones((3,3))
    dil_img = cv2.dilate(to_fill_img, kern, anchor=(1,1), iterations = 1)
    
    
    
    # Copy because dil_img needed at the end
    floodfill_img = dil_img.copy();
    # Mask used to flood filling, size 2 pixels bigger than the image.
    h, w = to_fill_img.shape
    mask = np.zeros((h+2, w+2), np.uint8) 
    # Floodfill from point (w-2,0)
    cv2.floodFill(floodfill_img, mask, seedPoint=(w-2,0), newVal=255);
    # Invert floodfilled image
    inv_floodfill_img = cv2.bitwise_not(floodfill_img)
    # Combine the two images to get the foreground.
    res_img = cv2.bitwise_or(dil_img, inv_floodfill_img)
    
    return res_img

def improve(images, threshold):
    #__PROCESSING OF THE IMAGES____________________________________________#
    numImg = 0
    avg0 = images[0][950:1500, 600:1200].mean()
    a = np.zeros(images.shape, dtype=np.uint8)
    for img in images :
        avg = img[950:1500, 600:1200].mean()
        # Filter the image (Gaussian way)
        #filter_img = cv2.GaussianBlur(img, (5,5), 0.3)
        filter_img = img
        # Threshold the image
        adp_th = threshold + int(avg-avg0);
        (T, thresh_img) = cv2.threshold(filter_img, adp_th ,255,cv2.THRESH_BINARY)    
        # Filling holes of the image 
        res_img = fill_holes(thresh_img, 2)
        print("numImg = %d, adaptive threshold = %d"%(numImg, adp_th))
        a[numImg] = res_img.astype(np.uint8)
        numImg += 1
    return a

def impr_less(images, threshold):
    #__PROCESSING OF THE IMAGES____________________________________________#
    numImg = 0
    a = np.zeros(images.shape, dtype=np.uint8)
    for img in images :
  
        # Filter the image (Gaussian way)
        #filter_img = cv2.GaussianBlur(img, (5,5), 0.3)
        # Threshold the image
        (T, thresh_img) = cv2.threshold(img, threshold ,255,cv2.THRESH_BINARY)    
        # Filling holes of the image 
        #res_img = fill_holes(thresh_img, 2)
    
        a[numImg] = thresh_img.astype(np.uint8) #res_img
        numImg += 1
    return a

thr = 220
res = improve(images, thr)
resl = impr_less(images, thr)
print('The shape of images is ', res.shape)
print('The type of image is ', res.dtype)

numImg = 0, adaptive threshold = 220
numImg = 1, adaptive threshold = 222
numImg = 2, adaptive threshold = 223
numImg = 3, adaptive threshold = 221
numImg = 4, adaptive threshold = 221
numImg = 5, adaptive threshold = 220
numImg = 6, adaptive threshold = 220
numImg = 7, adaptive threshold = 219
numImg = 8, adaptive threshold = 219
numImg = 9, adaptive threshold = 220
numImg = 10, adaptive threshold = 221
numImg = 11, adaptive threshold = 224
numImg = 12, adaptive threshold = 227
numImg = 13, adaptive threshold = 230
numImg = 14, adaptive threshold = 233
numImg = 15, adaptive threshold = 234
numImg = 16, adaptive threshold = 237
numImg = 17, adaptive threshold = 239
numImg = 18, adaptive threshold = 238
numImg = 19, adaptive threshold = 232
numImg = 20, adaptive threshold = 228
numImg = 21, adaptive threshold = 224
numImg = 22, adaptive threshold = 221
numImg = 23, adaptive threshold = 220
numImg = 24, adaptive threshold = 218
numImg = 25, adaptive threshold = 217
numImg = 26, adaptive 

In [8]:
num = 75
resize = tuple((np.array(images[0].shape)*0.3).astype(int))
im1 = cv2.resize(images[num], resize)
im2 = cv2.resize(res[num], resize)
im3 = cv2.resize(resl[num], resize)
cv2.imshow('original', im1)
cv2.imshow('improve', im2)
cv2.imshow('improve_reference', im3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
import tifffile
with tifffile.TiffWriter('out.tif') as tiff:
    for img in res:
        tiff.save(img)
print("Done")

Done
